In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models


import numpy as np
import wandb
import pandas as pd
import cv2
from tqdm import tqdm
import argparse
from PIL import Image

from sklearn.metrics import f1_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [59]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [60]:
class Model_VGG(nn.Module):
    def __init__(self, model_type):
        super(Model_VGG, self).__init__()

        if model_type == "vgg_16":
            self.model_backbone = models.vgg16(pretrained=True)
        elif model_type == "vgg_19":
            self.model_backbone = models.vgg19(pretrained=True)
        self.model_backbone.classifier[6] = nn.Linear(4096, 10)

    def forward(self, x):
        x = self.model_backbone(x)
        return x

In [61]:
class Digit_Dataset(Dataset):
    def __init__(self, df_data, transforms=None):
        self.data = df_data
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data.iloc[idx, 1:].values.reshape(28, 28).astype(np.uint8)
        # 3 channels
        img = np.stack((img,) * 3, axis=-1)
        label = self.data.iloc[idx, 0]
        if self.transforms:
            img = self.transforms(img)
        return img, label


In [62]:
class Training_Model:
    def __init__(self, model, optimizer, criterion, dict_dataloader):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.criterion = criterion
        self.train_loader = dict_dataloader["train_loader"]
        self.val_loader = dict_dataloader["val_loader"]
        self.best_f1_score = 0

    def train(self, epochs):
        print("============TRAINING START {}============".format(epochs))
        self.model.train()
        total_loss = 0
        preds, targets = [], []
        for batch_idx, (data, target) in enumerate(tqdm(self.train_loader)):
            data = data.to(device)
            target = target.to(device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target)
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            preds += predicted.tolist()
            targets += target.tolist()

        total_loss = round(total_loss / len(self.train_loader), 4)
        precision_scr = round(
            precision_score(targets, preds, average="macro"),
            4,
        )
        accuracy_scr = round(
            accuracy_score(targets, preds),
            4,
        )
        train_f1_scr = round(
            f1_score(targets, preds, average="macro"),
            4,
        )
        print(
            "train_loss: {}, train_precision: {}, train_accuracy: {}, train_f1_score: {}".format(
                total_loss, precision_scr, accuracy_scr, train_f1_scr
            )
        )
        wandb.log(
            {
                "train_loss": total_loss,
                "train_precision_score": precision_scr,
                "train_accuracy_score": accuracy_scr,
                "train_f1_score": train_f1_scr,
            }
        )

    def validation(self, epochs):
        print("============VAL START {}============".format(epochs))
        self.model.eval()
        total_loss = 0
        preds, targets = [], []
        for batch_idx, (data, target) in enumerate(tqdm(self.val_loader)):
            data = data.to(device)
            target = target.to(device)

            with torch.no_grad():
                output = self.model(data)
                loss = self.criterion(output, target)
                total_loss += loss.item()
                _, predicted = torch.max(output.data, 1)

                preds += predicted.tolist()
                targets += target.tolist()
        total_loss = round(total_loss / len(self.val_loader), 4)
        precision_scr = round(precision_score(targets, preds, average="macro"), 4)
        accuracy_scr = round(accuracy_score(targets, preds), 4)
        val_f1_scr = round(f1_score(targets, preds, average="macro"), 4)
        print(
            "val_loss: {}, val_precision: {}, val_accuracy: {}, val_f1_score: {}".format(
                total_loss, precision_scr, accuracy_scr, val_f1_scr
            )
        )
        wandb.log(
            {
                "val_loss": total_loss,
                "val_precision_score": precision_scr,
                "val_accuracy_score": accuracy_scr,
                "val_f1_score": val_f1_scr,
            }
        )
        if val_f1_scr > self.best_f1_score and val_f1_scr > 0.9:
            self.best_f1_score = val_f1_scr
            torch.save(
                self.model.state_dict(),
                "/kaggle/working/{}_f1_{}.pt".format("vgg_16", val_f1_scr),
            )
#             torch.jit.save(
#                 torch.jit.script(self.model),
#                 self.model.state_dict(),
#                 "/kaggle/working/{}_jit_f1_{}.pt".format("vgg_16", val_f1_scr),
#             )

In [63]:
def get_loader(csv_file, transforms, batch_size=64):
    dataset = Digit_Dataset(csv_file, transforms)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader


def get_input():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_type", type=str, default="vgg_16")
    parser.add_argument("--batch_size", type=int, default=8)
    parser.add_argument("--epochs", type=int, default=20)
    parser.add_argument("--lr", type=float, default=0.001)
    parser.add_argument("--seed", type=int, default=10)
    parser.add_argument("--weight_decay", type=float, default=0.0001)
    args = parser.parse_args()
    return args


def config_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

In [64]:
config_seed(10)

batch_size = 128
epochs = 50
lr = 0.001
weight_decay = 0.0001

data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

train_data, val_data = train_test_split(data, test_size=0.2, random_state=10)

transforms = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,)),
    ]
)

train_loader = get_loader(train_data, transforms=transforms, batch_size=batch_size)
val_loader = get_loader(val_data, transforms=transforms, batch_size=batch_size)

dict_dataloader = {"train_loader": train_loader, "val_loader": val_loader}
model_type = "vgg_16"
model = Model_VGG(model_type)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

wandb.init(
    project="Digit_Recognizer",
    group="VGG",
    name="{}_{}_{}_{}".format("vgg_16", batch_size, epochs, lr),
)

wandb.watch(model)

training_model = Training_Model(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    dict_dataloader=dict_dataloader,
)

for epoch in range(epochs):
    training_model.train(epoch)
    training_model.validation(epoch)

train_accuracy_score,▁▅█
train_f1_score,▁▅█
train_loss,█▄▁
train_precision_score,▁▅█
val_accuracy_score,▁▇█
val_f1_score,▁▇█
val_loss,█▃▁
val_precision_score,▁▇█
train_accuracy_score,0.9157
train_f1_score,0.9151
train_loss,0.3119


============TRAINING START 0============


100%|██████████| 263/263 [00:55<00:00,  4.77it/s]


train_loss: 2.1088, train_precision: 0.2039, train_accuracy: 0.2108, train_f1_score: 0.1985
============VAL START 0============


100%|██████████| 66/66 [00:07<00:00,  9.22it/s]
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

val_loss: 0.1061, val_precision: 0.9898, val_accuracy: 0.9899, val_f1_score: 0.9897
============TRAINING START 49============


100%|██████████| 263/263 [00:45<00:00,  5.75it/s]


train_loss: 0.0232, train_precision: 0.996, train_accuracy: 0.996, train_f1_score: 0.996
============VAL START 49============


100%|██████████| 66/66 [00:07<00:00,  9.00it/s]


val_loss: 0.172, val_precision: 0.9772, val_accuracy: 0.9755, val_f1_score: 0.9756


In [ ]:
model.load_state_dict(torch.load("/kaggle/working/vgg_16_f1_0.9902.pt"))

In [ ]:
model = model.to(device)

In [ ]:
DF_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
from torchvision import transforms
DF_submission = pd.DataFrame(columns=["ImageId", "Label"])
transforms = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,)),
    ]
)
for row in tqdm(range(DF_test.shape[0])):
    img = DF_test.iloc[row, :].values
    img = img.reshape(28, 28).astype(np.uint8)
    # 3 channel
    img = np.stack((img,) * 3, axis=-1)
    img = transforms(img)
    
    img = img.to(device)
    img = img.unsqueeze(0)
    with torch.no_grad():
        output = model(img)
        _, predicted = torch.max(output.data, 1)
        DF_test.iloc[row, :] = predicted.item()
        print(predicted.item())
        DF_submission = DF_submission.append( {"ImageId": row+1, "Label": predicted.item()}, ignore_index=True)
DF_submission.to_csv("/kaggle/working/vgg_16_f1_50_0.9915_submission.csv", index=False)
